In [ ]:
import pickle, numpy as np, pandas as pd
from sklearn.model_selection import StratifiedKFold,KFold,train_test_split


def generate_cv_splits(graph_dataset,out_pkl,f=0.7,n_split=10,y=None):
    datasets=pickle.load(open(graph_dataset,'rb'))
    IDs=datasets['df']['ID'].unique()
    if "component" in datasets['df'].columns:
        IDs=(datasets['df']['ID']+"_"+datasets['df']['component'].astype(str)).unique()
    idx=np.arange(len(datasets['graph_dataset']))

    X=pd.DataFrame(dict(idx=idx))
    cv_splits={}
    for i,(train_index, test_index) in enumerate((StratifiedKFold if not isinstance(y,type(None)) else KFold)(n_splits=n_split, random_state=42, shuffle=True).split(X['idx'],y)):
        cv_splits[i]=dict(train_idx=train_index,
                         test_idx=test_index)

    for i in range(len(cv_splits)):
        new_i=i+1
        if new_i==len(cv_splits):
            new_i=0
        X2=pd.DataFrame(dict(idx=cv_splits[new_i]['test_idx'],y=y[cv_splits[new_i]['test_idx']] if not isinstance(y,type(None)) else None))
        cv_splits[i]['val_idx']=train_test_split(X2['idx'],train_size=f,stratify=None if isinstance(y,type(None)) else y[X2['idx'].values])[0].values
        cv_splits[i]['train_idx']=np.setdiff1d(cv_splits[i]['train_idx'],cv_splits[i]['val_idx'])

    pickle.dump(cv_splits,open(out_pkl,'wb'))

In [ ]:
for k in ["your_data_set"]:
    generate_cv_splits(f"{k}/graph_datasets/imagenet_graph_data.pkl",f"{k}/graph_datasets/cv_splits.pkl",f=0.7,n_split=10,y=None)